# Query the data

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)
print(OPENAI_API_KEY)

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

### Helper function

In [ ]:
import json
def print_properties(item):
    print(
        json.dumps(
            item.properties,
            indent=2, sort_keys=True, default=str
        )
    )

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print_properties(item)

In [ ]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    return_metadata=MetadataQuery(distance=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

### Autocut

Return groups of results based on the quality/distance jumps

In [ ]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    auto_limit=1,
    return_metadata=MetadataQuery(distance=True)
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

## Filters

### Fetch with filters

In [ ]:
from weaviate.classes.query import Filter

wiki = client.collections.get("Wiki")

response = wiki.query.fetch_objects(
    limit=5,
    filters=Filter.by_property("title").like("music")
)

for item in response.objects:
    print_properties(item)

In [ ]:
from weaviate.classes.query import Filter

response = wiki.query.fetch_objects(
    limit=5,
    filters=Filter.by_property("title").like("m*ic") & Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
from weaviate.classes.query import Filter

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

## Keyword Search

[Docs - keyword/bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [ ]:
response = wiki.query.bm25(
    query="musical instruments",
    limit=5,
)

for item in response.objects:
    print_properties(item)

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title^3"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
)

for item in response.objects:
    print_properties(item)

### Hybrid - select properties

In [ ]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"]
)

for item in response.objects:
    print_properties(item)

### Hybrid - Explain score

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"],
    return_metadata=MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)
    print(item.metadata.explain_score)

## Close the client

In [ ]:
client.close()